# Bird's Selective Buzzer

## Abstract:

Birds like to come to the balcony. There are birds like the sunbird that come to suck the nectar from the flowers, swing and play on the plants and fly away. But, there are birds like the mynas and crows which are aggressive and terribly noisy (especially when they start nesting), and doves that leave a disastrous mess until their fledglings leave the nest.

***The problem:*** 
* How to drive away only the problematic birds?

***The solution:***
* A selective buzzer that buzzes only when an unwelcome bird arrives (dove, myna, and crow).

**Stage 1:** 
* Training a Machine learning model to classify the birds by their sounds.
* Binary classifying the birds (buzzer activator or not).

**Stage 2:**
* Building a buzzer that makes noise when a problematic bird arrives.
* Sending the order to activate according to the results of the classification.

***Available data:***
* Birds' sounds from [xeno-canto.org](https://xeno-canto.org).

***Potential users:***
* Anyone who wants to drive away birds selectively from the balcony, building, field etc.

## Work plan

### Prepare the data:
* Import audio files from [xeno-canto.org](https://xeno-canto.org).
* Preprocessing the audio files using [Audacity](https://www.audacityteam.org): 
    - Cut silent parts in the beginning of the audio file.
    - Duplicate parts in the short files to have duration of at least 3 sec.
    - Reduce background noise.
    - Expand the amplitude envelope where needed.
    - *note:* The original files are recorded in different channel mode, sample rate, duration, quality, environments etc. 
* Format the files as *.wav* - The processing audio libraries use wav format.
* Build a dataframe of the files, short file name (index) and the bird's class.
* Load the audio files set to equal channels (mono), sample rate (44100), and duration (3 sec).


### EDA
* Visualize some samples of each type.
* Check the data
* Set the parameters for comparison.
* Encode the categorcial features.
* Scale the numeric features (train only) and transform it to all sets.
* Check outliers.

### Models:
* Due to the low amount of samples - a neural network cannot be used. The training will be done in traditional ML, using feature engineering.
* Baseline model - A constant (True).
* Logistic regression
* Decision tree classifier (with upsample, downsample and threshold)
* Random forest
* Cross validation

### Conclusions
* Overall conclusions.

### Run the model on a Raspberry Pi





## Inisialization:

### Libraries

In [ ]:
# Libraries
import pandas as pd
import numpy as np
from pydub import AudioSegment
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import scipy as sp
import scipy.fft as spf
from scipy.io import wavfile
from scipy import signal
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import sklearn.metrics as skm 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import ipywidgets as widgets #buttons for contitional execution
from ipywidgets import interact, interact_manual
import librosa
import librosa.display
import IPython.display as ipd
from joblib import dump

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Constants

In [ ]:
# Constants
LOCAL_PATH = 'G:\Eliana\Documents\Data Science\CA\Project booster\\'
LOCAL_PATH_MP3 = LOCAL_PATH + 'mp3\\' # path to the mp3 folder
LOCAL_PATH_WAV = LOCAL_PATH + 'wav\\' # path to the wav folder
BIRDS = ['sunbird', 'dove', 'myna', 'crow'] # the species of the birds
RAND_STATE = 12 # random state
SR=44100 # sample rate=44100

### Buttons for conditional executes

In [ ]:
# create execution buttons
btn_conv = widgets.Button(description = 'Convert files to wav') # 'Convert files to .wav'
btn_time = widgets.Button(description = 'Plot waves') # 'Plot waves'

output = widgets.Output()

def on_conv_clicked(b):
    convert_mp3_to_wav()
    
def on_btn_time_clicked(b):
    plot_waves()
    
    
# on_click events:
btn_conv.on_click(on_conv_clicked)
btn_time.on_click(on_btn_time_clicked)

### Convert mp3 files to wav

In [ ]:
# Files in mp3 directory

files = os.listdir(LOCAL_PATH + 'mp3\\')

In [ ]:
# Convert mp3 files to wav. 
def convert_mp3_to_wav():
    """Convert the mp3 files to wav. The fuction is executed by pressing the button below, as there's no need
    to repeat it once the files are converted and there are no changes in the original mp3s"""
    
    print('converting...')
    for file in files:
        src = LOCAL_PATH_MP3 + file
        dst = LOCAL_PATH_WAV + file.replace('mp3', 'wav')

        try:
            # convert mp3 to wav                                                            
            sound = AudioSegment.from_mp3(src)
            wav_file=sound.export(dst, format="wav")
            wav_file.close()      
        except:
            print('Error converting', file)
            continue

# diaplay the execution button            
display(btn_conv, output) 

### List the files

In [ ]:
# List the files
files_wav = os.listdir(LOCAL_PATH_WAV)
files_wav

### Functions

#### Load audio files

In [ ]:
# Load audio files
def load_audio(path, filename):
    # load 3 seconds of the file at sample rate=44100, mono.
    signal, sr = librosa.load(path + filename, sr=SR, mono=True, duration=3)
    return signal, sr

# vectorize the function
vec_load_audiot = np.vectorize(load_audio)

#### Extract the audio index

In [ ]:
# Extract the audio index

def extract_file_index(string):
    return string.lower().split(' ')[0]

# vectorize the function
vec_extract_file_index = np.vectorize(extract_file_index)

#### Extract the bird species from the filename

In [ ]:
# Extract the bird species from the filename

def extract_bird_name(string):
    words = string.lower().split(' ')
    bird=''
    
    for word in words:
        if word in BIRDS:
            bird = word

    if bird=='':
        bird='unknown'
    return bird

# vectorize the function
vec_extract_bird_name = np.vectorize(extract_bird_name)

#### Class Bird

* The data contain arrays and lists, and cannot be held in a dataframe cell. The class is used to hold the data per bird.
* The index of the bird is equal to the index in the dataframe.

In [ ]:
class Bird:
    def __init__(self, file_index, name):
        self.name = name
        self.file_index = file_index
        self.signal = [] # the audio file raw data
        self.sr = 0 # sample_rate
        self.sc = [] # spectral centroid
        self.stft = [] # short time fourier transform
        self.Y = [] # spectrogram
        self.loud_freqs = [] # the frequencies of the higher eights part of the db.
        self.loudest_freq = 0
        

### Create a dataframe

In [ ]:
# Create a dataframe
df = pd.DataFrame({'file':files_wav})
df

In [ ]:
# add a short file name and the bird species
df['file_index'] = vec_extract_file_index(df['file'])
df['bird']= vec_extract_bird_name(df['file'])
df

### Create birds objects and load the data 

In [ ]:
# create bird object

def create_bird(indx, name, path, filename):
    bird = Bird(indx, name)
    bird.signal, bird.sr = load_audio(path, filename)
    return bird

# vectorize the function
vec_create_bird = np.vectorize(create_bird)

In [ ]:
# create an array of 'Bird' objects.
birds = vec_create_bird(df['file_index'], df['bird'], LOCAL_PATH_WAV, df['file'])
len(birds)

In [ ]:
# species proportion
species_prop = df.groupby(['bird'])['file'].count().reset_index()
species_prop.columns = ['bird', 'count']
species_prop['percent'] = ((species_prop['count'] / df.shape[0]) *100).round(2)
species_prop

In [ ]:
# plt the graph
fig = go.Figure(data=[go.Pie(labels=species_prop['bird'], values=species_prop['percent'])])
fig.update_layout(title="Proportions of the various species of birds")
fig.show() 
#fig.write_image("images/propbird.png") # used to save the image

### Prepare the data

#### Basic information of the audio files (all have the same sample rate, duration and no. of channels)

In [ ]:
for i in df.index:
    df.loc[i, 'shape'] = birds[i].signal.shape
df

In [ ]:
# verify that all the signals have the same shape
print(df['shape'].min(), df['shape'].max())

In [ ]:
# duration in seconds of 1 sample
sample_duration = 1 / SR
print(f"One sample lasts for {sample_duration:6f} seconds")

In [ ]:
# total number of samples in audio file (=shape)
tot_samples = len(birds[0].signal)  
tot_samples

### EDA

***Note:***

The audio data is processed in two domains: *Time* and *Frequency*.
- In the time domain we can see the amplitude changes over time. But, we have no information about the frequencies involved.
- In the frequency domain we can see the frequencies involved. But we don't know when each frequency appear.
- In order to combine the information from the two domains - we use spectrograms, which are like a heat map that show us the changes of the magnitude of each frequency over time.
- As both scales of frequencies and magnitude (in decibels) are logarithmic, the values of the parameters will be mostly in power of 2.

#### Time domain

In [ ]:
# plot data in time domain
#counter=0 # used for saving the plots with unique name
def plot_waves():
    df_shape = 5  # first 5 birds
    #df.shape[0] # - for all the birds

    plt.figure(figsize=(15, 30))

    for i in range(df_shape):
        plt.subplot(df_shape, 1, i+1)
        librosa.display.waveshow(birds[i].signal, alpha=0.5)
        plt.ylim((-1, 1))
        plt.title(birds[i].name)
    #    plt.savefig("images/time"+birds[i].name+str(counter)+".png") # used to save the image
    #    counter+=1

    plt.show()
    
# diaplay the execution button            
display(btn_time, output)

###### Conclusion:
Although there are cases of repetitive patterns, there's not enough data to classify the birds.

#### Extracting Short-Time Fourier Transform

In [ ]:
FRAME_SIZE = 2048
HOP_SIZE = 512

##### Visualizing the spectrogram

In [ ]:
# plot a spectrogram

def plot_spectrogram(Y, sr, hop_length, y_axis="linear", title='', counter=0):
    plt.figure(figsize=(25, 10))
    librosa.display.specshow(Y, 
                             sr=sr, 
                             hop_length=hop_length, 
                             x_axis="time", 
                             y_axis=y_axis,
                            cmap='coolwarm')
    
    plt.title(title)
    plt.colorbar(format="%+2.f")
#    plt.savefig("images/spec"+title+str(counter)+".png") # used to save the images
    

##### Visualising spectrograms from different birds

In [ ]:
frames = range(len(birds[0].signal))
t = librosa.frames_to_time(frames, hop_length=HOP_SIZE)

In [ ]:
#plot=False # calculate only
plot=True # calculate and plot the spectrograms

counter=0 # used for saving the plots with unique name

for b in birds:
    # extract Short-Time Fourier Transform
    S_b = librosa.stft(b.signal, n_fft=FRAME_SIZE, hop_length=HOP_SIZE)

    # calculate the spectrogram
    Y_b = librosa.power_to_db(np.abs(S_b) ** 2)  #Y(m,k) = |S(m,k)|^2

    if plot:
        # plot the spectrogram
        plot_spectrogram(Y_b, SR, HOP_SIZE, y_axis="log", title=b.name, counter=counter)

    counter+=1
    # save to bird object
    b.stft = S_b
    b.Y = Y_b

As seen in the spectrograms, the typical loudest frequencies are about 256 Hz and higher (the dove has the lowest frequencies). So, we can assume that all the frequencies below that are background noise, and will be excluded.


#### Add features

In [ ]:
%%time
# find the loudest frequencies (the highest eighth)
for b in birds:
    hi=b.Y.max()-((b.Y.max()-b.Y.min())/8)
    b.loud_freqs= (np.unique(np.where(b.Y > hi)[0])+1) * 16
    b.loudest_freq = (np.where(b.Y == b.Y.max())[0]+1) * 16

In [ ]:
%%time
# add features
for i in df.index:
    df.loc[i, 'hi_lim'] = birds[i].loud_freqs.max()
    df.loc[i, 'lo_lim'] = np.maximum(birds[i].loud_freqs.min(), 192)
    df.loc[i, 'loudest_freq'] = birds[i].loudest_freq
df

#### Encode the categorical features

In [ ]:
bird_dict = {'sunbird': 0, 'dove': 1, 'myna': 2, 'crow': 3}
df['bird_code'] = df['bird'].map(bird_dict)
df['buzz'] = df['bird_code'].astype(bool)
df

#### Split the data

In [ ]:
# split data into training, test and validation 
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RAND_STATE)
df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=RAND_STATE)

#### Scale the numeric features

*Note:*

In the last version of features, there are only frequency based features, so there's no need to scale them. 

In [ ]:
numeric = ['hi_lim', 'lo_lim', 'loudest_freq']

#### Check Outliers (train only)

In [ ]:
Q1 = df_train[numeric].quantile(0.25)
Q3 = df_train[numeric].quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
# check if the value is an outlier
(df_train[numeric] < (Q1 - 1.5 * IQR)) | (df_train[numeric] > (Q3 + 1.5 * IQR))

In [ ]:
df[numeric][df[numeric]==True].sum()

###### Conclusion:
* There are no relevant outliers.

In [ ]:
cols_to_drop = ['file', 'file_index', 'bird', 'bird_code', 'buzz', 'shape']

In [ ]:
# declare variables for features and target feature 

X_train = df_train.drop(cols_to_drop, axis=1)
y_train = df_train['buzz']
X_valid = df_valid.drop(cols_to_drop, axis=1)
y_valid = df_valid['buzz']
X_test = df_test.drop(cols_to_drop, axis=1)
y_test = df_test['buzz']

print(X_train.shape)
print(X_valid.shape) 
print(X_test.shape) 

## Models

### Functions:

In [ ]:
def predict_and_score(model, X_valid, y_valid):
    predicted_valid = model.predict(X_valid)
    precision = skm.precision_score(y_valid, predicted_valid)
    recall = skm.recall_score(y_valid, predicted_valid)
    
    print('accuracy_score', skm.accuracy_score(y_valid, predicted_valid).round(3)) 
    print('f1_score', skm.f1_score(y_valid, predicted_valid).round(3))    
    print('Recall:', recall.round(3))
    print('Precision:', precision.round(3))


### Base line model (constant True)

In [ ]:
# valid
print('f1_score', skm.f1_score(y_valid, np.ones(y_valid.shape)).round(3)) 

In [ ]:
# test
print('f1_score', skm.f1_score(y_test, np.ones(y_test.shape)).round(3)) 

### Logistic regression

In [ ]:
# fit, predict and score logistic regression
model = LogisticRegression(random_state=RAND_STATE, solver='liblinear', multi_class='auto')
lg = model.fit(X_train, y_train)
predict_and_score(lg, X_valid, y_valid)

# save the model
dump(model, 'bird-lg.joblib')

In [ ]:
#test log_reg
predict_and_score(lg, X_test, y_test)

###### Conclusion:
* The validation set has worse results than just guessing.
* The test set has much better results than the base line model.

### DecisionTreeClassifier

In [ ]:
# upsample
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

# shuffle 
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled 

In [ ]:
# downsample
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones]) 
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones]) 
    features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [ ]:
#fit_f1 (model.fit and calculate F1)
#* basic: (model, features_train, target_train)
#* upsample: (model,features_upsampled, target_upsampled)
#* downsample: (model, features_downsampled, target_downsampled)

def fit_f1(model, X_train, y_train, X_val_tst, y_val_tst):
    model.fit(X_train, y_train)
    predicted_valid = model.predict(X_val_tst)

    print('F1:', skm.f1_score(y_val_tst, predicted_valid).round(3)) # binary
    return model 

In [ ]:
for i in range(1,10):
    model = DecisionTreeClassifier(random_state=RAND_STATE, class_weight='balanced', max_depth=i)
    print(i, ':') 
    dtc= fit_f1(model, X_train, y_train, X_valid, y_valid)

In [ ]:
# validate the model with optimized hyper parameters
model = DecisionTreeClassifier(random_state=RAND_STATE, class_weight='balanced', max_depth=1)
dtc= fit_f1(model, X_train, y_train, X_valid, y_valid)
predict_and_score(dtc, X_valid, y_valid)

# save the model
dump(dtc, 'bird-dtc.joblib')

In [ ]:
#test DecisionTreeClassifier
predict_and_score(dtc, X_test, y_test)

###### Conclusions:
* The F1 score is higher than just guessing - both the valid and the test sets.
* The perfect precision shows that the model doesn't drive away any welcome bird, but the recall shows that there are cases in which an unwelcome bird is not driven away.

#### Upsample

In [ ]:
X_upsampled, y_upsampled = upsample(X_train, y_train, 4)
print(X_upsampled.shape)
print(y_upsampled.shape)

In [ ]:
dtc = fit_f1(model, X_upsampled, y_upsampled, X_valid, y_valid)
predict_and_score(dtc, X_valid, y_valid)

# save the model
dump(dtc, 'bird-dtc-up.joblib')

In [ ]:
predict_and_score(dtc, X_test, y_test)

###### Conclusions:
* The perfect precision shows that the model doesn't drive away any welcome bird, but the recall shows that there are cases in which an unwelcome bird is not driven away.

#### Downsample

In [ ]:
X_downsampled, y_downsampled = downsample(X_train, y_train, 0.8)

print(X_downsampled.shape)
print(y_downsampled.shape)

In [ ]:
# valid
dtc=fit_f1(model,X_downsampled, y_downsampled, X_valid, y_valid)
predict_and_score(dtc, X_valid, y_valid)

# save the model
dump(dtc, 'bird-dtc-down.joblib')

In [ ]:
# test
dtc=fit_f1(model,X_downsampled, y_downsampled, X_test, y_test)
predict_and_score(dtc, X_test, y_test)

###### Conclusions:
* With downsampling we got perfect results in the test set. But, since there are relatively few samples, it has to be checked on much larger amount of samples before celebrating.

#### Threshold (on the Decision-Tree)

In [ ]:
# threshold - valid
model.fit(X_train, y_train)
probabilities_valid = model.predict_proba(X_valid)
probabilities_one_valid = probabilities_valid[:, 1]

for threshold in np.arange(0, 0.9, 0.1):
    predicted_valid = probabilities_one_valid > threshold 
    precision = skm.precision_score(y_valid, predicted_valid)
    recall = skm.recall_score(y_valid, predicted_valid)


    print('Threshold = {:.2f} | Precision = {:.3f}, Recall = {:.3f}, F1 = {:.3f}'.format(
        threshold, precision, recall, (2 * precision * recall/(precision + recall))))

In [ ]:
# threshold - test
model.fit(X_train, y_train)
probabilities_test = model.predict_proba(X_test)
probabilities_one_test = probabilities_test[:, 1]

threshold = 0.2
predicted_test = probabilities_one_test > threshold 
precision = skm.precision_score(y_test, predicted_test)
recall = skm.recall_score(y_test, predicted_test)


print('Threshold = {:.2f} | Precision = {:.3f}, Recall = {:.3f}, F1 = {:.3f}'.format(
        threshold, precision, recall, (2 * precision * recall/(precision + recall))))

###### Conclusions:
* The perfect precision shows that the model doesn't drive away any welcome bird, but the recall shows that there are cases in which an unwelcome bird is not driven away.

### ROC curve

In [ ]:
# ROC curve
fpr, tpr, thresholds = skm.roc_curve(y_test, probabilities_one_test)

plt.figure()

plt.plot(fpr, tpr)

# ROC curve for random model (looks like a straight line)
plt.plot([0, 1], [0, 1], linestyle='--')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

plt.title("ROC curve")
plt.show()

###### Conclusion:
* The ROC curve high results.

### RandomForestClassifier

In [ ]:
for i in range(5, 50, 5):
    model = RandomForestClassifier(random_state=RAND_STATE, n_estimators=i)
    rf= fit_f1(model, X_train, y_train, X_valid, y_valid)
    print(i)

In [ ]:
# test RandomForestClassifier
model = RandomForestClassifier(random_state=RAND_STATE, n_estimators=5)
rf= fit_f1(model, X_train, y_train, X_test, y_test)

# save the model
dump(rf, 'bird-rf.joblib')

###### Conclusions:
* Running the test set results perfect F1 score (better than the valid). In this case, even low number of estimators gives perfect results.

### Cross validation

In [ ]:
data = df
features = data.drop(cols_to_drop, axis=1)
target = data['buzz']

model = DecisionTreeClassifier(random_state=RAND_STATE)

# calculate scores by calling cross_value_score function 
scores=cross_val_score(model, features, target, cv=10) 
final_score=sum(scores)/len(scores)
print('Average model evaluation score:', final_score.round(3))

# save the model
dump(model, 'bird-cv.joblib')

###### Conclusions:
* The CV model has lower results than the tree's model, and not much higher than the base line. but, it is less sensitive to variance in the sets as it shuffles the samples.

## Conclusions

* Looking at the results - the perfect model for this case is a Decision tree classifier with downsampling. It has perfect results.
* Considering  that there are only about a hundred samples, the result should be taken as limited - but there's a potential for further research.

## Run the model on a Raspberry Pi
* Run scarecrow.py on the Raspberry Pi

## References

* [xeno-canto.org](https://xeno-canto.org).
* Videos by Valerio Velardo - 'The Sound of AI'